In [1]:
import functools

In this tutorial we will learn how to write decorators to increase the functionality of a given function. Then we will provide some real world examples on how they can be useful to speed up computations and are super powerful by helping the develoeprs in avoiding a lot of overhead by this added functionality. In a way it is like inception as we are defining a function inside a function.

In [2]:
#Let us start with a simple decorator
from datetime import datetime


def my_decorator(function_as_argument):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
           value = function_as_argument()
        else:   
           value = f'Sleep time ' + function_as_argument()
        return value   
    return wrapper 

def say_kapoorlabs():
    return f'We are at Kapoorlabs.'

#decorated call only returns the print statement when the if condition is satisfied, a direct call to the function will not 
#obey such a condition
say_kapoorlabs_decorator = my_decorator(say_kapoorlabs)

print(say_kapoorlabs_decorator(),say_kapoorlabs())



We are at Kapoorlabs. We are at Kapoorlabs.


In [3]:
#Syntactic Sugar

# By using this syntax of a decorator the function can be directly called and will get the deocrate functionality
@my_decorator
def say_kapoorlabs():
      print('We are at Kapoorlabs')

say_kapoorlabs()     


We are at Kapoorlabs


In [4]:
#Passing arguments into the decorator

def my_decorator_argument(function_as_argument):
    def wrapper(*args, **kwargs):
        if 2 <= datetime.now().hour < 22:
           value = function_as_argument(*args, **kwargs)
        else:   
           value = f'Sleep time now, ' + function_as_argument(*args, **kwargs)
        return value   
    return wrapper 

@my_decorator_argument
def say_kapoorlabs_argument(start, end):
      return f'We are at Kapoorlabs from {start} till {end} hours'


assign_value = say_kapoorlabs_argument(2,22)
print(assign_value, say_kapoorlabs_argument, help(say_kapoorlabs_argument))

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

We are at Kapoorlabs from 2 till 22 hours <function my_decorator_argument.<locals>.wrapper at 0x000001DA23327B80> None


In [5]:
#What about functools? (gets the true location of the function and not locates it as wrapped inside a decorator)
def my_decorator_argument(function_as_argument):
    @functools.wraps(function_as_argument)
    def wrapper(*args, **kwargs):
        if 2 <= datetime.now().hour < 22:
           value = function_as_argument(*args, **kwargs)
        else:   
           value = f'Sleep time now, ' + function_as_argument(*args, **kwargs)
        return value   
    return wrapper 
@my_decorator_argument
def say_kapoorlabs_argument(start, end):
      return f'We are at Kapoorlabs from {start} till {end} hours'


assign_value = say_kapoorlabs_argument(2,22)
print(assign_value, say_kapoorlabs_argument, help(say_kapoorlabs_argument))

Help on function say_kapoorlabs_argument in module __main__:

say_kapoorlabs_argument(start, end)

We are at Kapoorlabs from 2 till 22 hours <function say_kapoorlabs_argument at 0x000001DA233770D0> None


In [6]:
#Real world examples
#Our broiler plate

def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        value = func(*args, **kwargs)
        return value
    return wrapper    




In [7]:
import time

def timer(func):
    """ Print the runtime of the decorated function """
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time 
        print(f'Finished  {func.__name__!r} in {run_time:.4f}secs')
        return value
    return wrapper_timer  

In [8]:
@timer
def waste_some_time(num_times):

    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [9]:
waste_some_time(10)

Finished  'waste_some_time' in 0.0203secs


In [10]:
def debug(func):
    """ Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f'{k} = {v!r}' for k, v in kwargs.items()]
        signature = ','.join(args_repr + kwargs_repr)
        print(f'Calling {func.__name__} ({signature})')
        value = func(*args, **kwargs)
        return value
    return wrapper_debug    

In [11]:
@debug
def make_greeting(name, age = None):

    if age is None:
       return f'{name}'
    else:
       return f'age is just a number {age}'   

In [14]:
from functools import lru_cache


@timer
@lru_cache(maxsize=3)
@debug
def fibonacci_numbers(number):

    if number == 0 or number == 1:
        return 1

    return fibonacci_numbers(number - 1 ) + fibonacci_numbers(number - 2)    

In [15]:
answer_fib = fibonacci_numbers(100)

Calling fibonacci_numbers (100)
Calling fibonacci_numbers (99)
Calling fibonacci_numbers (98)
Calling fibonacci_numbers (97)
Calling fibonacci_numbers (96)
Calling fibonacci_numbers (95)
Calling fibonacci_numbers (94)
Calling fibonacci_numbers (93)
Calling fibonacci_numbers (92)
Calling fibonacci_numbers (91)
Calling fibonacci_numbers (90)
Calling fibonacci_numbers (89)
Calling fibonacci_numbers (88)
Calling fibonacci_numbers (87)
Calling fibonacci_numbers (86)
Calling fibonacci_numbers (85)
Calling fibonacci_numbers (84)
Calling fibonacci_numbers (83)
Calling fibonacci_numbers (82)
Calling fibonacci_numbers (81)
Calling fibonacci_numbers (80)
Calling fibonacci_numbers (79)
Calling fibonacci_numbers (78)
Calling fibonacci_numbers (77)
Calling fibonacci_numbers (76)
Calling fibonacci_numbers (75)
Calling fibonacci_numbers (74)
Calling fibonacci_numbers (73)
Calling fibonacci_numbers (72)
Calling fibonacci_numbers (71)
Calling fibonacci_numbers (70)
Calling fibonacci_numbers (69)
Calling

In [16]:
#Finally a decorator that also passes in variables , here we have to define an outer function with an inner decorated function
from psygnal import Signal
def change_handler(*widgets, init=True):
        def decorator_change_handler(handler):
            @functools.wraps(handler)
            def wrapper(*args):
                source = Signal.sender()
                emitter = Signal.current_emitter()
                if debug:
                   
                    print(f'{str(emitter.name).upper()}: {source.name} = {args!r}')
                return handler(*args)

            for widget in widgets:
                widget.changed.connect(wrapper)
                if init:
                    widget.changed(widget.value)
            return wrapper

        return decorator_change_handler

# In this case the decorator is the @change_handler that takes in widget type as an argument and emits a signal if use changes a GUI element
